In [1]:
import pandas as pd

ratings = pd.read_json('./ratings.jsonl', lines=True)
content = pd.read_json('./content.jsonl', lines=True)
targets = pd.read_csv('targets.csv', encoding='latin-1', sep=',')

In [2]:
_content = content[['ItemId', 'Genre', 'imdbVotes']]

# Discarding the pipes between the plot keywords' and getting only the first five words
# df_1['plot_keywords'] = df_1['plot_keywords'].map(lambda x: x.split('|')) # 'Plot' field but only the keywords
# df_1['plot_keywords'] = df_1['plot_keywords'].apply(lambda x: " ".join(x)) #  'Plot' field but only the keywords

# Discarding the pipes between the genres 
_content['Genre'] = _content['Genre'].map(lambda x: x.lower().replace(', ', ' '))
# _content['Genre'] = _content['Genre'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(',')))
# _content['Features'] = _content['Actors'] +' '+ _content['Director'] +' '+ _content['Genre']
# _content['Genre'] = _content['Genre'].str.replace(r'[\']', '', regex=True)
# _content['Genre'] = _content['Genre'].str.replace(' ', '', regex=True)
_content['Features'] = _content['Genre']

_content['imdbVotes'] = _content['imdbVotes'].str.replace('N/A', '1', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].str.replace(',', '', regex=True)
_content['imdbVotes'] = _content['imdbVotes'].astype(str).astype('float64')
_content['imdbVotes'] = _content['imdbVotes'].fillna(0)

/tmp/ipykernel_8559/2044994005.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Genre'] = _content['Genre'].apply(lambda x: ' '.join(x.replace(' ', '').lower() for x in x.split(',')))
/tmp/ipykernel_8559/2044994005.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _content['Features'] = _content['Genre']
/tmp/ipykernel_8559/2044994005.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [3]:
import pandas as pd
class Rocchio:

    def __init__(self, ratings_df):
        self.__ratings = ratings_df[['UserId', 'ItemId', 'Rating']].copy()
        self.__ratings['Rating'] = self.__ratings.apply(lambda row: self.__rescaleRating(row['Rating']), axis=1)

    def fit(self, features_df):
        self.__features = self.__ratings.copy()

        # Também pode ser compreendido como a computação da matriz de utilidade dos itens.
        self.__split_features_into_columns(features_df)
        self.__compute_user_utility()

    def predict(self, user_item_df):
        self.targets = user_item_df[['ItemId', 'UserId']].copy()
        self.targets = self.targets.merge(self.__features, on='ItemId', how='outer')
        self.targets = self.targets.fillna(0)

        self.targets = pd.merge(self.targets, self.__user_utility, on='UserId')

        consine_sim = self.targets[['UserId', 'ItemId']]
        consine_sim['user_DOT_item'] = 0
        consine_sim['user_distance'] = 0
        consine_sim['item_distance'] = 0

        for feat in self.__features_list:
            consine_sim['user_DOT_item'] += self.targets['u_' + feat] * self.targets[feat]
            consine_sim['user_distance'] += self.targets['u_' + feat] * self.targets['u_' + feat]
            consine_sim['item_distance'] += self.targets[feat]        * self.targets[feat]

        consine_sim['user_distance'] = consine_sim['user_distance'].pow(1./2)
        consine_sim['item_distance'] = consine_sim['item_distance'].pow(1./2)
        consine_sim['Similarity']    = consine_sim['user_DOT_item'] / \
                                            (consine_sim['user_distance'] * consine_sim['item_distance'])

        # consine_sim = consine_sim.drop(['user_DOT_item', 'user_distance', 'item_distance'], axis=1)

        return consine_sim

    def __split_features_into_columns(self, features_df):
        self.__features = features_df[['ItemId', 'Features']].copy()
        splitted_features = self.__features['Features'].str.split(' ')
        self.__features_list = list(dict.fromkeys([y for x in splitted_features for y in x]).keys())

        for feat in self.__features_list:
            self.__features[feat] = self.__features['Features'].apply(lambda x: 1 if feat in x else 0)

        self.__features = self.__features.drop(['Features'], axis=1)

    def __rescaleRating(self, old_rating):
        if old_rating == 10:
            return 3
        elif old_rating > 7:
            return 2
        elif old_rating > 5:
            return 1
        return 0

    def __compute_user_utility(self):
        self.__user_utility = self.__ratings.copy()
        self.__user_utility = self.__user_utility.merge(self.__features, on='ItemId')

        self.__user_utility['Frequency'] = self.__user_utility.groupby('UserId')['UserId'].transform('count')

        for feat in self.__features_list:
            self.__user_utility[feat] = self.__user_utility[feat] * self.__user_utility['Rating'] / self.__user_utility['Frequency']

        self.__user_utility = self.__user_utility.drop(['ItemId', 'Frequency', 'Rating'], axis=1)
        self.__user_utility = self.__user_utility.groupby('UserId').sum()

        for feat in self.__features_list:
            self.__user_utility['u_' + feat] = self.__user_utility[feat]
            self.__user_utility = self.__user_utility.drop(feat, axis=1)

In [4]:
rocchio = Rocchio(ratings)
rocchio.fit(_content[['ItemId', 'Features']])

In [5]:
rocchio_similarities = rocchio.predict(targets)

/tmp/ipykernel_8559/4161733725.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consine_sim['user_DOT_item'] = 0
/tmp/ipykernel_8559/4161733725.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consine_sim['user_distance'] = 0
/tmp/ipykernel_8559/4161733725.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [6]:
rocchio.targets.head()
print(rocchio.targets.columns)

Index(['ItemId', 'UserId', 'documentary', 'short', 'news', 'sport', 'horror',
       'comedy', 'action', 'adventure', 'drama', 'fantasy', 'romance',
       'history', 'crime', 'family', 'war', 'n/a', 'biography', 'sci-fi',
       'mystery', 'thriller', 'western', 'animation', 'music', 'musical',
       'film-noir', 'adult', 'talk-show', 'reality-tv', 'game-show',
       'u_documentary', 'u_short', 'u_news', 'u_sport', 'u_horror', 'u_comedy',
       'u_action', 'u_adventure', 'u_drama', 'u_fantasy', 'u_romance',
       'u_history', 'u_crime', 'u_family', 'u_war', 'u_n/a', 'u_biography',
       'u_sci-fi', 'u_mystery', 'u_thriller', 'u_western', 'u_animation',
       'u_music', 'u_musical', 'u_film-noir', 'u_adult', 'u_talk-show',
       'u_reality-tv', 'u_game-show'],
      dtype='object')


In [7]:
recommendations = rocchio_similarities
recommendations = recommendations.merge(_content[['ItemId', 'imdbVotes']], on='ItemId')
recommendations['SimilarityIMDB'] = recommendations['Similarity']*recommendations['imdbVotes']
recommendations = recommendations.sort_values(['UserId', 'SimilarityIMDB', 'imdbVotes'], ascending=[True, False, False])
recommendations.head()


,UserId,ItemId,user_DOT_item,user_distance,item_distance,Similarity,imdbVotes,SimilarityIMDB
2546,0006246bee,ce297c0305,1.090909,0.851833,1.732051,0.739390,411358.0,304153.843249
1741,0006246bee,9277d31cc9,1.163636,0.851833,1.732051,0.788682,352366.0,277904.820956
2434,0006246bee,c1ee6829f5,0.890909,0.851833,1.732051,0.603835,430209.0,259775.196494
2237,0006246bee,b273133e05,0.636364,0.851833,1.732051,0.431311,557708.0,240545.384895
204,0006246bee,1e5bdbcb76,0.636364,0.851833,1.732051,0.431311,475339.0,205018.760194


In [8]:
output_file = open('out-20.csv','w')
print('UserId,ItemId', file=output_file)
pd.set_option("display.max_rows", None, "display.max_columns", None)
for row in recommendations.itertuples():
    print('{},{}'.format(row.UserId, row.ItemId), file=output_file)